In [2]:
source("utils/plot.R")

In [3]:
options(repr.plot.width=4, repr.plot.height=3, jupyter.plot_scale=1)

In [4]:
parameters <- read_tsv(snakemake@input$parameters, col_types=cols()) %>%
    select(uuid, selection_coefficient)

In [5]:
sel_breaks <- 10^(c(-2, -1.5, -1.0, -0.5, 0, 0.5, 1.0, 1.5, 2))

### Selection strength part

In [8]:
selstrength_raw <- read_tsv(snakemake@input$selstrength, col_types=cols()) %>%
    inner_join(parameters, on=uuid)

In [9]:
selstrength <- selstrength_raw %>%
    mutate(selection_bracket=cut(selection_coefficient, breaks=sel_breaks, dig.lab=3)) %>%
    group_by(selection_bracket) %>%
    summarize(
        rmse=rmse(true_log_selection_coefficient, predicted_log_selection_coefficient),
        mean_relative_error=mean_relative_error(true_log_selection_coefficient, predicted_log_selection_coefficient)
    )

In [10]:
selection_bracket_levels <- levels(selstrength$selection_bracket)

In [11]:
baseline <- tibble(
    selection_bracket="All",
    rmse=rmse(selstrength_raw$true_log_selection_coefficient, selstrength_raw$predicted_log_selection_coefficient),
    mean_relative_error=mean_relative_error(selstrength_raw$true_log_selection_coefficient, selstrength_raw$predicted_log_selection_coefficient)
)

In [12]:
selstrength <- bind_rows(selstrength, baseline) %>%
    mutate(
        selection_bracket=factor(selection_bracket, levels=c(selection_bracket_levels, 'All')),
        selection_bracket_label=str_replace_all(selection_bracket, c(
            "\\("="",
            "\\]"="",
            ","="-"
        )),
        selection_bracket_label=fct_reorder(selection_bracket_label, as.integer(selection_bracket)),
    )

In [13]:
sel_fig <- ggplot(selstrength) +
    geom_col(aes(x=selection_bracket_label, y=1/mean_relative_error, fill=selection_bracket_label=="All")) +
    scale_fill_manual(values=c('darkgrey', 'darkred')) +
    coord_flip() +
    labs(
        title = "Selection strength",
        y = "1/Mean relative error",
        x = "Selection coefficient"
    ) +
    sweeps_theme +
    theme(
        legend.position='none'
    )

In [14]:
sel_fig

### Sweep mode part

In [16]:
sweepmode_raw <- read_tsv(snakemake@input$sweepmode, col_types=cols()) %>%
    inner_join(parameters, on=uuid)

In [17]:
sweepmode <- sweepmode_raw %>%
    mutate(selection_bracket=cut(selection_coefficient, breaks=sel_breaks, dig.lab=3)) %>%
    group_by(selection_bracket) %>%
    summarize(accuracy=accuracy(true_ix, predicted_ix))

In [18]:
selection_bracket_levels <- levels(sweepmode$selection_bracket)

In [19]:
baseline <- tibble(
    selection_bracket="All",
    accuracy=accuracy(sweepmode_raw$true_ix, sweepmode_raw$predicted_ix)
)

In [20]:
sweepmode <- bind_rows(sweepmode, baseline) %>%
    mutate(
        selection_bracket=factor(selection_bracket, levels=c(selection_bracket_levels, 'All')),
        selection_bracket_label=str_replace_all(selection_bracket, c(
            "\\("="",
            "\\]"="",
            ","="-"
        ))
    )

In [21]:
sweepmode_fig <- ggplot(sweepmode) +
    geom_col(aes(x=selection_bracket_label, y=accuracy, fill=selection_bracket_label=="All")) +
    scale_fill_manual(values=c('darkgrey', 'darkred')) +
    ylim(0, 1) +
    coord_flip() +
    labs(
        title = "Sweep mode",
        y = "Accuracy",
        x = "Selection coefficient"
    ) +
    sweeps_theme +
    theme(
        legend.position='none',
        axis.title.y=element_blank(),
        axis.text.y=element_blank()
    )

In [22]:
sweepmode_fig

### Plot them together

In [23]:
all_fig <- plot_grid(
    sel_fig, sweepmode_fig,
    nrow=1,
    axis='tb',
    align='t',
    rel_widths=c(1.3, 1)
)

In [24]:
sweeps_save(snakemake@output[[1]], all_fig, asp=1.8)